<a href="https://colab.research.google.com/github/AlexanderCardarasUCSC/EEG-Decoding-to-Images/blob/main/CSE_247_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ThoughtViz

**Complete the following steps before running the cells below**

1.
Change runtime to GPU 

>Runtime > Change runtime type > GPU

2.
For each of the following files, add a Google Drive shortcut to your Drive root folder(My Drive):

>https://drive.google.com/file/d/1atP9CsjWIT-hg3fX--fcC1hg0uvg9bEH/view

>https://drive.google.com/file/d/1x32IulYeBVmkshEKweijMX3DK1Wu8odx/view

>https://drive.google.com/file/d/1cq8RTBiwqO-Jo0TZjBNlRHZEhjKDknKP/view

>https://drive.google.com/file/d/1U9qtN1SlOS3dzd2BwWWHhJiMz_0yNW9U/view

>https://drive.google.com/file/d/1uFFhvTsU2nmdaecR2WPWsiGJfgI3as1_/view


**Mount drive and download project**

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
!git clone "https://github.com/ptirupat/ThoughtViz.git" "/content/ThoughtViz"

Cloning into '/content/ThoughtViz'...
remote: Enumerating objects: 105, done.
remote: Total 105 (delta 0), reused 0 (delta 0), pack-reused 105
Receiving objects: 100% (105/105), 28.01 KiB | 9.34 MiB/s, done.
Resolving deltas: 100% (47/47), done.


**Unzip project files**

In [3]:
!unzip -q "/content/gdrive/My Drive/data.zip" -d "/content/ThoughtViz" 
!unzip -q "/content/gdrive/My Drive/images.zip" -d "/content/ThoughtViz/training"
!unzip -q "/content/gdrive/My Drive/eeg_models.zip" -d "/content/ThoughtViz/models" 
!unzip -q "/content/gdrive/My Drive/trained_classifier_models.zip" -d "/content/ThoughtViz/training"
!unzip -q "/content/gdrive/My Drive/gan_models.zip" -d "/content/ThoughtViz/models"
!echo "Download Finished!"

Download Finished!


**Update outdated module calls and make code compatible with colab environment**

In [1]:
# Update Keras module structure
!sed -i "s/keras.layers/keras/" ThoughtViz/layers/mog_layer.py

# Change System Path
!sed -i "1i import sys\nsys.path.insert(0,'/content/ThoughtViz')\n" /content/ThoughtViz/testing/test.py
!sed -i "s/\..\//\/content\/ThoughtViz\//" ThoughtViz/testing/test.py

# Rename GAN model directory to match code
!mv /content/ThoughtViz/models/gan_models/thoughtviz/ /content/ThoughtViz/models/gan_models/final/

sed: can't read ThoughtViz/layers/mog_layer.py: No such file or directory
sed: can't read /content/ThoughtViz/testing/test.py: No such file or directory
sed: can't read ThoughtViz/testing/test.py: No such file or directory
mv: cannot stat '/content/ThoughtViz/models/gan_models/thoughtviz/': No such file or directory


**Test code**

Note: Images are not saved by default. 
>To save the output image, you must edit the *ThoughtViz/testing/test.py* file and add the line **img.save('output.png')** after *img.show()* in the test_deligan_final function. 

In [8]:
!python3 ThoughtViz/testing/test.py

2021-04-04 01:22:18.458431: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2021-04-04 01:22:20.272168: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-04 01:22:20.273032: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-04 01:22:20.307834: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retu

# EEG decoding to images

**Complete the following steps before running the cells below**

1.
Change runtime to GPU 

>Runtime > Change runtime type > GPU

2.
For each of the following files, add a Google Drive shortcut to your Drive root folder(My Drive):

>https://drive.google.com/file/d/1atP9CsjWIT-hg3fX--fcC1hg0uvg9bEH/view

>https://drive.google.com/file/d/1x32IulYeBVmkshEKweijMX3DK1Wu8odx/view

>https://drive.google.com/file/d/1cq8RTBiwqO-Jo0TZjBNlRHZEhjKDknKP/view

>https://drive.google.com/file/d/1U9qtN1SlOS3dzd2BwWWHhJiMz_0yNW9U/view

>https://drive.google.com/file/d/1uFFhvTsU2nmdaecR2WPWsiGJfgI3as1_/view


**Mount drive**

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

**Unzip project files**

In [4]:
!unzip -q "/content/gdrive/My Drive/data.zip" -d "/content/ThoughtViz" 
!unzip -q "/content/gdrive/My Drive/images.zip" -d "/content/ThoughtViz/training"
!unzip -q "/content/gdrive/My Drive/eeg_models.zip" -d "/content/ThoughtViz/models" 
!unzip -q "/content/gdrive/My Drive/trained_classifier_models.zip" -d "/content/ThoughtViz/training"
!unzip -q "/content/gdrive/My Drive/gan_models.zip" -d "/content/ThoughtViz/models"
!echo "Download Finished!"

unzip:  cannot find or open /content/gdrive/My Drive/data.zip, /content/gdrive/My Drive/data.zip.zip or /content/gdrive/My Drive/data.zip.ZIP.
unzip:  cannot find or open /content/gdrive/My Drive/images.zip, /content/gdrive/My Drive/images.zip.zip or /content/gdrive/My Drive/images.zip.ZIP.
unzip:  cannot find or open /content/gdrive/My Drive/eeg_models.zip, /content/gdrive/My Drive/eeg_models.zip.zip or /content/gdrive/My Drive/eeg_models.zip.ZIP.
unzip:  cannot find or open /content/gdrive/My Drive/trained_classifier_models.zip, /content/gdrive/My Drive/trained_classifier_models.zip.zip or /content/gdrive/My Drive/trained_classifier_models.zip.ZIP.
unzip:  cannot find or open /content/gdrive/My Drive/gan_models.zip, /content/gdrive/My Drive/gan_models.zip.zip or /content/gdrive/My Drive/gan_models.zip.ZIP.
Download Finished!


**LSTM stuff**

In [5]:
# Code goes here #

**GAN stuff**

In [6]:
# Code goes here #

**Test Code**

In [7]:
# Code goes here #